In [ ]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data

Checking dataset data/OpportunityUCIDataset.zip
... dataset path data/OpportunityUCIDataset.zip not found
... creating directory data
... downloading data from https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat


In [1]:
%connect_info

{
  "stdin_port": 40867, 
  "ip": "127.0.0.1", 
  "control_port": 52943, 
  "hb_port": 58644, 
  "signature_scheme": "hmac-sha256", 
  "key": "f5162548-fcbc-4766-b4ce-1bce16b3b436", 
  "kernel_name": "", 
  "shell_port": 35481, 
  "transport": "tcp", 
  "iopub_port": 54399
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /run/user/1000/jupyter/kernel-1948e908-9be0-46e5-b196-e0d700761cf1.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [66]:
import numpy as np
import cPickle
from sliding_window import sliding_window
import keras as K
from keras.utils.np_utils import to_categorical
from sklearn import metrics

def load_dataset(filename):
    f = file(filename, 'rb')
    data = cPickle.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

def opp_sliding_window_x(data_x, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    return data_x.astype(np.float32)

def opp_sliding_window_y(data_y, ws, ss):
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_y.reshape(len(data_y)).astype(np.uint8)

# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

NUM_CLASSES = [5,6,14,24,14,24,18]



In [82]:
len(x_train)

46495

In [92]:
x_train, y_train, x_test, y_test = load_dataset('data/oppChallenge_gestures.data')

print len(x_train), len(y_train)
y_train = [y_train[:,i] for i in range(len(NUM_CLASSES))]
y_test = [y_test[:,i] for i in range(len(NUM_CLASSES))]

# Sensor data is segmented using a sliding window mechanism --- TEST!
x_test = opp_sliding_window_x(x_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
for id in range(len(NUM_CLASSES)):
    y_test[id] = opp_sliding_window_y(y_test[id], SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    print(" ..after sliding window (testing): inputs {0}, targets {1}".format(x_test.shape, y_test[id].shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
x_test = x_test.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
print "FIRST"

# Sensor data is segmented using a sliding window mechanism --- TRAIN!
x_train = opp_sliding_window_x(x_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
for id in range(len(NUM_CLASSES)):
    y_train[id] = opp_sliding_window_y(y_train[id], SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    print(" ..after sliding window (training): inputs {0}, targets {1}".format(x_train.shape, y_train[id].shape))

for i in xrange(len(NUM_CLASSES)):
    y_train[i] = to_categorical(y_train[i], NUM_CLASSES[i])
    y_test[i] = to_categorical(y_test[i], NUM_CLASSES[i])

# Data is reshaped since the input of the network is a 4 dimension tensor -- TRAIN!
x_train = x_train.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
557963 557963
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
FIRST
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
 ..after sliding window (training): 

In [94]:
print x_train.shape, y_train[0].shape

(46495, 1, 24, 113) (46495, 5)


In [117]:
from keras import backend
import sklearn.metrics as metrics

    
def run_model(model, id, epochs=5, repeat=6):
    results = []
    for it in range(repeat):
        model.fit(x_train, y_train[id], nb_epoch=epochs, batch_size=100)
        y_predict = model.predict(x_test)
        f1_measure = metrics.f1_score(np.array(map(np.argmax, y_test[id])),
                                      np.array(map(np.argmax, y_predict)),
                                      range(NUM_CLASSES[id]), average='weighted')
        print "(epoch %i) F1 measure: %s" % ((it + 1) * epochs , str(f1_measure))
        results.append([(it + 1) * epochs, f1_measure])
    return results
#confusion = metrics.confusion_matrix(np.array(map(np.argmax, y_test)), np.array(map(np.argmax, y_predict)))

In [105]:
models = []
for id in range(len(NUM_CLASSES)):
    gold = K.models.Sequential()
    gold.add(K.layers.convolutional.Convolution2D(5, 5, 1, border_mode='same', activation='relu', input_shape=(1,24,113)))
    gold.add(K.layers.convolutional.Convolution2D(5, 5, 1, border_mode='same', activation='relu'))
    gold.add(K.layers.convolutional.Convolution2D(5, 5, 1, border_mode='same', activation='relu'))
    gold.add(K.layers.convolutional.Convolution2D(5, 5, 1, border_mode='same', activation='relu'))
    print gold.output_shape
    gold.add(K.layers.Permute((2,1,3)))
    gold.add(K.layers.Reshape((24, 5 * 113)))
    gold.add(K.layers.recurrent.LSTM(128, return_sequences=True, dropout_W=0.4, dropout_U=0.4))
    gold.add(K.layers.recurrent.LSTM(128, dropout_U=0.3))
    gold.add(K.layers.Dense(NUM_CLASSES[id], activation='softmax'))
    print gold.output_shape
    gold.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    models.append(gold)

(None, 5, 24, 113)
(None, 5)
(None, 5, 24, 113)
(None, 6)
(None, 5, 24, 113)
(None, 14)
(None, 5, 24, 113)
(None, 24)
(None, 5, 24, 113)
(None, 14)
(None, 5, 24, 113)
(None, 24)
(None, 5, 24, 113)
(None, 18)


In [113]:
print y_train[0].shape, models[0].output_shape

(46495, 5) (None, 5)


In [118]:
results = []
for id in range(len(NUM_CLASSES)):
    print "Type id: %i" % id
    result = run_model(models[id], id, epochs=4, repeat=7)
    results.append(result)
    

Type id: 0
Epoch 1/4
46495/46495 [==============================] - 183s - loss: 0.2730 - acc: 0.8783   
Epoch 2/4
46495/46495 [==============================] - 180s - loss: 0.1642 - acc: 0.9351   
Epoch 3/4
46495/46495 [==============================] - 180s - loss: 0.1352 - acc: 0.9475   
Epoch 4/4
46495/46495 [==============================] - 180s - loss: 0.1212 - acc: 0.9526   
(epoch 4) F1 measure: 0.857555752649
Epoch 1/4
46495/46495 [==============================] - 180s - loss: 0.1132 - acc: 0.9562   
Epoch 2/4
46495/46495 [==============================] - 180s - loss: 0.1060 - acc: 0.9594   
Epoch 3/4
46495/46495 [==============================] - 180s - loss: 0.1005 - acc: 0.9606   
Epoch 4/4
46495/46495 [==============================] - 180s - loss: 0.0968 - acc: 0.9626   
(epoch 8) F1 measure: 0.873592863376
Epoch 1/4
46495/46495 [==============================] - 180s - loss: 0.0925 - acc: 0.9638   
Epoch 2/4
46495/46495 [==============================] - 180s - loss:

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(epoch 4) F1 measure: 0.730684556729
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0436 - acc: 0.9834   
Epoch 2/4
46495/46495 [==============================] - 181s - loss: 0.0421 - acc: 0.9840   
Epoch 3/4
46495/46495 [==============================] - 181s - loss: 0.0410 - acc: 0.9843   
Epoch 4/4
46495/46495 [==============================] - 181s - loss: 0.0404 - acc: 0.9847   
(epoch 8) F1 measure: 0.739887079661
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0395 - acc: 0.9851   
Epoch 2/4
46495/46495 [==============================] - 181s - loss: 0.0392 - acc: 0.9852   
Epoch 3/4
46495/46495 [==============================] - 181s - loss: 0.0389 - acc: 0.9852   
Epoch 4/4
46495/46495 [==============================] - 181s - loss: 0.0380 - acc: 0.9857   
(epoch 12) F1 measure: 0.763425350699
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0376 - acc: 0.9858   
Epoch 2/4
46495/46495 [===================

In [119]:
print results

[[[4, 0.85755575264920625], [8, 0.87359286337588138], [12, 0.87635505842055006], [16, 0.87880300175266657], [20, 0.88454373399760255], [24, 0.88253803302375], [28, 0.88275593571427302]], [[4, 0.71537550389496707], [8, 0.73938375667731848], [12, 0.74313905189821494], [16, 0.7453440138347841], [20, 0.74479456703752223], [24, 0.75758154821122969], [28, 0.74745525588598838]], [[4, 0.73068455672877464], [8, 0.73988707966142553], [12, 0.76342535069933004], [16, 0.74505142552798898], [20, 0.7525272330911722], [24, 0.75199246697912725], [28, 0.75825259560316571]], [[4, 0.6896727314509612], [8, 0.70602241819461065], [12, 0.71315245051811826], [16, 0.72723186715602151], [20, 0.7315695965043022], [24, 0.74502286604293577], [28, 0.72525155507294148]], [[4, 0.70991510919511613], [8, 0.72669543704218842], [12, 0.72755274097008771], [16, 0.72127115242262096], [20, 0.72648525592285318], [24, 0.72991825928897114], [28, 0.73541745876579434]], [[4, 0.63849728168656927], [8, 0.68227986802104823], [12, 0.6

In [120]:
results2 = []
for id in range(len(NUM_CLASSES)):
    print "Type id: %i" % id
    result = run_model(models[id], id, epochs=4, repeat=7)
    results2.append(result)

Type id: 0
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0696 - acc: 0.9742   
Epoch 2/4
46495/46495 [==============================] - 181s - loss: 0.0679 - acc: 0.9745   
Epoch 3/4
46495/46495 [==============================] - 181s - loss: 0.0688 - acc: 0.9739   
Epoch 4/4
46495/46495 [==============================] - 181s - loss: 0.0684 - acc: 0.9744   
(epoch 4) F1 measure: 0.891640127089
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0696 - acc: 0.9734   
Epoch 2/4
46495/46495 [==============================] - 181s - loss: 0.0685 - acc: 0.9740   
Epoch 3/4
46495/46495 [==============================] - 181s - loss: 0.0689 - acc: 0.9742   
Epoch 4/4
46495/46495 [==============================] - 181s - loss: 0.0670 - acc: 0.9747   
(epoch 8) F1 measure: 0.888309759913
Epoch 1/4
46495/46495 [==============================] - 181s - loss: 0.0674 - acc: 0.9746   
Epoch 2/4
46495/46495 [==============================] - 181s - loss:

In [134]:
for i in range(14):
    sys.stdout.write(str(i * 4 + 4) + ", ")
    for j in range(7):
        sys.stdout.write(str(merged[j][i][1]) +", ")
    print ""

4, 0.857555752649, 0.715375503895, 0.730684556729, 0.689672731451, 0.709915109195, 0.638497281687, 0.850714565488, 
8, 0.873592863376, 0.739383756677, 0.739887079661, 0.706022418195, 0.726695437042, 0.682279868021, 0.880729514535, 
12, 0.876355058421, 0.743139051898, 0.763425350699, 0.713152450518, 0.72755274097, 0.6790146745, 0.884724993959, 
16, 0.878803001753, 0.745344013835, 0.745051425528, 0.727231867156, 0.721271152423, 0.685224456832, 0.89595443574, 
20, 0.884543733998, 0.744794567038, 0.752527233091, 0.731569596504, 0.726485255923, 0.688795969603, 0.893390923982, 
24, 0.882538033024, 0.757581548211, 0.751992466979, 0.745022866043, 0.729918259289, 0.694643073159, 0.898641010316, 
28, 0.882755935714, 0.747455255886, 0.758252595603, 0.725251555073, 0.735417458766, 0.696727017228, 0.899293057127, 
32, 0.891640127089, 0.758706962712, 0.763933453246, 0.731765907951, 0.747003752746, 0.699422555079, 0.900916710814, 
36, 0.888309759913, 0.745077908297, 0.759586330399, 0.739317069622, 0.

In [130]:
import sys